In [1]:
!pip install --upgrade spacy
!pip install --upgrade gensim
!pip install --upgrade scipy
!pip install --upgrade pyemd
!pip install --upgrade sklearn
!pip install --upgrade nltk
!pip install --upgrade datasets
!pip install --upgrade jupyter
!pip install --upgrade ipywidgets
!pip install --upgrade POT
!pip install --upgrade keras
!pip install torch

!python -m spacy download en_core_web_sm

     --------------------------------------- 12.8/12.8 MB 17.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import spacy
from spacy.lang.en import English
import numpy as np
import gensim
from gensim.models import KeyedVectors
import gensim.downloader as api
from scipy.spatial.distance import cosine, jensenshannon, euclidean, cityblock
from scipy.stats import spearmanr
from pyemd import emd
from sklearn.metrics.pairwise import cosine_similarity, paired_distances, euclidean_distances
import nltk
from nltk.corpus import stopwords
import string
from datasets import load_dataset
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Embedding, Lambda, Subtract, Dense
from keras.models import Model
from keras.optimizers import RMSprop
import torch
import torch.nn as nn
import torch.optim as optim
from nltk.stem import WordNetLemmatizer
import time

c:\Users\andre\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
# Load spacy model
nlp = spacy.load('en_core_web_sm')

# Load word2vec model
w2v_model = api.load('word2vec-google-news-300')

# Define stop words
nltk.download('stopwords')
stop_words = stopwords.words('english') + list(string.punctuation)

dataset = load_dataset("stsb_multi_mt", 'en')
train_dataset = dataset['train'].to_pandas()
test_dataset = dataset['test'].to_pandas()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Found cached dataset stsb_multi_mt (C:/Users/andre/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Preprocess text using spaCy
def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    return ' '.join(tokens)

In [5]:
# Compute distance between texts using WMD
def compute_wmd_distance(text1, text2):
    tokenized_text1 = text1.split()
    tokenized_text2 = text2.split()
    try:
        distance = w2v_model.wmdistance(tokenized_text1, tokenized_text2)
        return distance
    except ValueError:
        return 0

In [6]:
# Compute distance between texts using cosine distance
def compute_cosine_distance(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    vector1 = doc1.vector
    vector2 = doc2.vector
    try:
        distance = cosine(vector1, vector2)
        return distance
    except ValueError:
            return 0

In [7]:
# Compute distance between texts using Jensen-Shannon divergence
def compute_jensen_shannon_distance(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    vector1 = doc1.vector
    vector2 = doc2.vector
    try:
        distance = jensenshannon(vector1, vector2)
        return distance
    except ValueError:
        return 0

In [8]:
# Compute distance between texts using Euclidean distance
def compute_euclidean_distance(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    vector1 = doc1.vector
    vector2 = doc2.vector
    try:
        distance = euclidean(vector1, vector2)
        return distance
    except ValueError:
        return 0

In [9]:
# Compute distance between texts using Manhattan distance
def compute_manhattan_distance(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    vector1 = doc1.vector
    vector2 = doc2.vector
    try:
        distance = cityblock(vector1, vector2)
        return distance
    except ValueError:
        return 0

In [10]:
# Compute distance between texts using Earth Mover's Distance
def compute_emd_distance(text1, text2):
    tokenized_text1 = text1.split()
    tokenized_text2 = text2.split()
    distance_matrix = np.zeros((len(tokenized_text1), len(tokenized_text2)))
    for i, word1 in enumerate(tokenized_text1):
        if word1 in stop_words:
            continue
        if word1 not in w2v_model.index_to_key:
            continue
        for j, word2 in enumerate(tokenized_text2):
            if word2 in stop_words:
                continue
            if word2 not in w2v_model.index_to_key:
                continue
            distance_matrix[i, j] = w2v_model.distance(word1, word2)
    distance = distance_matrix.sum()
    return distance

In [11]:
preprocessed_texts1 = test_dataset['sentence1'].apply(preprocess_text)
preprocessed_texts2 = test_dataset['sentence2'].apply(preprocess_text)

In [12]:
start_time = time.time()
wmd_distances = preprocessed_texts1.combine(preprocessed_texts2, compute_wmd_distance)
wmd_time = time.time() - start_time
start_time = time.time()
cosine_distances = preprocessed_texts1.combine(preprocessed_texts2, compute_cosine_distance)
cosine_time = time.time() - start_time
start_time = time.time()
jensen_distances = preprocessed_texts1.combine(preprocessed_texts2, compute_jensen_shannon_distance)
jensen_time = time.time() - start_time
start_time = time.time()
euclidean_distances_ = preprocessed_texts1.combine(preprocessed_texts2, compute_euclidean_distance)
euclidean_time = time.time() - start_time
start_time = time.time()
manhattan_distances = preprocessed_texts1.combine(preprocessed_texts2, compute_manhattan_distance)
manhattan_time = time.time() - start_time
start_time = time.time()
emd_distances = preprocessed_texts1.combine(preprocessed_texts2, compute_emd_distance)
emd_time = time.time() - start_time

c:\Users\andre\anaconda3\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)


In [13]:
wmd_correlation, _ = spearmanr(wmd_distances, test_dataset['similarity_score'])
cosine_correlation, _ = spearmanr(cosine_distances, test_dataset['similarity_score'])
jensen_correlation, _ = spearmanr(jensen_distances, test_dataset['similarity_score'])
euclidean_correlation, _ = spearmanr(euclidean_distances_, test_dataset['similarity_score'])
manhattan_correlation, _ = spearmanr(manhattan_distances, test_dataset['similarity_score'])
emd_correlation, _ = spearmanr(emd_distances, test_dataset['similarity_score'])

In [14]:
print("WMD correlation:", wmd_correlation, "Time:", wmd_time)
print("Cosine similarity correlation:", cosine_correlation, "Time:", cosine_time)
print("Jensen correlation:", jensen_correlation, "Time:", jensen_time)
print("Euclidean correlation:", euclidean_correlation, "Time:", euclidean_time)
print("Manhattan correlation:", manhattan_correlation, "Time:", manhattan_time)
print("EMD correlation:", emd_correlation, "Time:", emd_time)

wmd_correlation = abs(wmd_correlation)
cosine_correlation = abs(cosine_correlation)
jensen_correlation = abs(jensen_correlation)
euclidean_correlation = abs(euclidean_correlation)
manhattan_correlation = abs(manhattan_correlation)
emd_correlation = abs(emd_correlation)

# Determine the best approach based on the highest correlation coefficient
max_correlation = max(wmd_correlation, cosine_correlation, jensen_correlation, euclidean_correlation, manhattan_correlation, emd_correlation)
if max_correlation == wmd_correlation:
    print("WMD distance is the best approach.")
elif max_correlation == cosine_correlation:
    print("Cosine similarity distance is the best approach.")
elif max_correlation == jensen_correlation:
    print("Jensen distance is the best approach.")
elif max_correlation == euclidean_correlation:
    print("Euclidean distance is the best approach.")
elif max_correlation == manhattan_correlation:
    print("Manhattan distance is the best approach.")
else:
    print("EMD distance is the best approach.")

WMD correlation: -0.6442835036826644 Time: 7.004554271697998
Cosine similarity correlation: -0.4875207713658226 Time: 8.993525505065918
Jensen correlation: -0.01335479678178198 Time: 8.319007396697998
Euclidean correlation: -0.4857486668831016 Time: 8.140328407287598
Manhattan correlation: -0.48537166617169974 Time: 8.086314916610718
EMD correlation: 0.08835242356436084 Time: 145.2808859348297
WMD distance is the best approach.


In [15]:
# Compute distance between texts using WMD
def compute_wmd_distance_siamese(preprocessed_text1, preprocessed_text2):
    tokenized_text1 = preprocessed_text1.split()
    tokenized_text2 = preprocessed_text2.split()
    try:
        distance = w2v_model.wmdistance(tokenized_text1, tokenized_text2)
        return distance
    except ValueError:
        return 0

In [16]:
from tqdm import tqdm
samples = []
labels = []
for row in tqdm(test_dataset.iterrows()):
    left_words_tmp = preprocess_text(row[1][0])
    left_words = [w2v_model.key_to_index[word] if word in w2v_model.index_to_key else 0 for word in left_words_tmp]
    right_words_tmp = preprocess_text(row[1][1])
    right_words = [w2v_model.key_to_index[word] if word in w2v_model.index_to_key else 0 for word in right_words_tmp]
    wmd_distance = compute_wmd_distance_siamese(left_words_tmp, right_words_tmp)
    samples.append([left_words, right_words])
    labels.append(wmd_distance)

1379it [09:23,  2.45it/s]


In [17]:
max_ = 50
for i in range(len(samples)):
    len_ = len(samples[i][0])
    if len_ > max_:
        samples[i][0] = samples[i][0][: max_]
    else:
        samples[i][0].extend([0] * (max_ - len_))
        
    len_ = len(samples[i][1])
    if len_ > max_:
        samples[i][1] = samples[i][1][: max_]
    else:
        samples[i][1].extend([0] * (max_ - len_))

In [18]:
def euclidean_distance(inputs):
    """
    Calculates the Euclidean distance between two tensors.
    """
    x, y = inputs
    distance = tf.sqrt(tf.reduce_sum(tf.square(x - y), axis=1, keepdims=True))
    return distance

In [19]:
input_shape = (max_,)
input_left = tf.keras.Input(shape=input_shape, dtype='int32', name='input_left')
input_right = tf.keras.Input(shape=input_shape, dtype='int32', name='input_right')
embedding_layer = tf.keras.layers.Embedding(input_dim=len(w2v_model.index_to_key), output_dim=300, input_length=max_)
lstm_layer = tf.keras.layers.LSTM(150)
left = lstm_layer(embedding_layer(input_left))
right = lstm_layer(embedding_layer(input_right))
distance_layer = tf.keras.layers.Lambda(euclidean_distance)([left, right])
output_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(distance_layer)
model = tf.keras.Model(inputs=[input_left, input_right], outputs=output_layer)
model.compile(optimizer='adam', loss='mse')

In [20]:
x = [np.array([sample[0] for sample in samples], dtype=np.int32), np.array([sample[1] for sample in samples], dtype=np.int32)]
y = np.array(labels, dtype=np.float32)

In [ ]:
# Train the model on the sample data
model.fit(x=x, y=y, batch_size=32, epochs=1)

In [21]:
def compute_wmd_distance_test(tokens1, tokens2):
    try:
        distance = w2v_model.wmdistance(tokens1, tokens2)
        return distance
    except ValueError:
        return 0
    
tokens1all = [text.split() for text in preprocessed_texts1]
tokens2all = [text.split() for text in preprocessed_texts2]

In [33]:
start_time = time.time()
for data in zip(tokens1all, tokens2all):
    wmd_distances = compute_wmd_distance_test(data[0], data[1])
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.40190625190734863 seconds ---


In [23]:
x1 = []
x2 = []
for text1, text2 in zip(preprocessed_texts1, preprocessed_texts2):
    tokenized_text1 = text1.split()
    tokenized_text2 = text2.split()
    
    tmp = [w2v_model.key_to_index[word] if word in w2v_model.index_to_key else 0 for word in tokenized_text1]
    len_ = len(tmp)
    if len_ > max_:
        tmp = tmp[: max_]
    else:
        tmp.extend([0] * (max_ - len_))
    x1.append(tmp)
    
    tmp = [w2v_model.key_to_index[word] if word in w2v_model.index_to_key else 0 for word in tokenized_text2]
    len_ = len(tmp)
    if len_ > max_:
        tmp = tmp[: max_]
    else:
        tmp.extend([0] * (max_ - len_))
    x2.append(tmp)
    

In [38]:
start_time = time.time()
wmd_distances = model.predict([np.array(x1), np.array(x2)])
print("--- %s seconds ---" % (time.time() - start_time))

44/44 [==============================] - 1s 18ms/step
--- 0.8751041889190674 seconds ---
